# Load Dataset

https://huggingface.co/datasets/doc2dial#data-fields


In [89]:
from datasets import load_dataset

split = "train"
cache_dir = "./data_cache"

#User's turn: utterance= question, reference=grounding document span_id, can be empty, "precondition"/"solution" are the
#actual grounding spans
#Gold label for grounding
dialogue_dataset = load_dataset(
    "doc2dial",
    name="dialogue_domain",  # this is the name of the dataset for the second subtask, dialog generation
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

document_dataset = load_dataset(
    "doc2dial",
    name="document_domain",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642)
Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642)


Insights from looking at the data structure:
For training I will need the following data:
From Dialogue:
doc_id - Grounding document
turns.utterance -> Dialogue history
references.sp_id -> ID for the output text_sp

From Documents:
To get output span:
doc_id -> spans -> id_sp -> text_sp column is the actual output span


# Naive Bayes with simple word counts
Task:
- input:
    - dialogue history for current discussion (turns.utterance from dialogue)
    - doc_id for the grounding document to find the span from
- output:
    - the text_sp in documents for a certain doc_id that the agent would need to respond to the user

Training:
- fit(X, y) -> X (n_samples, n_features) training vectors, y  = target values (e.g relevant=1, not relevant)


In [95]:
#For Input:
#Utterances:
#for each dialogue with the same doc id: get users utterance into a list: if role=user -> document.turns.utterance
# -> rows = number of utterances
# use all dialogues with the same grounding document for training (e.g one classifier per grounding doc)

#Vectorise the input: (word counts)
# rows=utterances, columns = vocabulary, value=wordcount

#Labels:
# there are multiple spans per utterances
# rows = utterance, columns = span_ids for the document, cell = 1 or 0 depending on if spans is relevant or not
# document->turn->references->sp_ids

import pandas as pd
#just take the first doc id for the moment
doc_id = dialogue_dataset[0]['doc_id']

#get all dialogues for that doc_id
dialogues_for_doc_id = dialogue_dataset.filter(lambda doc: doc['doc_id'] == doc_id)

#get all data spans for that doc
document_for_id = document_dataset.filter(lambda doc: doc['doc_id'] == doc_id)
# getting the spans for that doc id
spans_df_for_id = [pd.json_normalize(span) for span in document_for_id['spans']][0]
spans =





Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642/cache-9274aaa595ce18fe.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642/cache-2ab233a5254763ad.arrow


In [ ]:
#Train on the document spans
#Input X_train: rows=spans, columns=feature for span + utternace for that span
#Output Y: 0 if span relevant or not

# Evaluation of Simplest Model

Required output format is a json file of predictions with id, prediction_text and no_answer_probability
https://github.com/doc2dial/sharedtask-dialdoc2021/blob/master/scripts/sample_files/sample_prediction_subtask1.json


# Iteration


- extend for training data set as suggested by dialdoc https://mrqa.github.io/2019/shared